# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

In [1]:
resource_group = "fbakscpurg" # Feel free to modify these
aks_name = "fbAKSClustercpu"
location = "eastus"

image_name = "fboylu/kerasres152-cpu" 
selected_subscription = "'Team Danielle Internal'"

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [6]:
!az login -o table

In [7]:
!az account set --subscription $selected_subscription

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [10]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [11]:
# !az group create --name $resource_group --location $location

Below, we create the AKS cluster in the resource group we created earlier.

In [46]:
%%time
# !az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_D4_v2

CPU times: user 0 ns, sys: 6 µs, total: 6 µs
Wall time: 9.78 µs


### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [3]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [4]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustercpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [5]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-1   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-2   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-3   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-4   Ready     agent     9d        v1.9.6


Let's check the pods on our cluster.

In [8]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          9d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          9d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          9d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          9d
kube-system   kube-proxy-225hz                        1/1       Running   0          9d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          9d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          9d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          9d
kube-system   kube-proxy-tvnpv                        1/1       Running   0          9d
kube-system   kube-svc-redirect-278jk                 1/1       Running   0          9d
kube-system   kube-svc-redirect

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [9]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerasres152-cpu",

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [10]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [11]:
write_json_to_file(app_template, 'az-dl.json')

In [12]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [13]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "fboylu/kerasres152-cpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
       

Next, we will use kubectl create command to deploy our application.

In [14]:
!kubectl create -f az-dl.json

deployment.apps "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [18]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-56bf5dd85c-drkb4               1/1       Running   0          11m
default       azure-dl-56bf5dd85c-knnkq               1/1       Running   0          11m
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          9d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          9d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          9d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          9d
kube-system   kube-proxy-225hz                        1/1       Running   0          9d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          9d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          9d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          9d
kube-system   kube-proxy-tvnp

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [19]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND      SUBOBJECT                   TYPE      REASON                SOURCE                              MESSAGE
59m         59m          1         azure-dl-56bf5dd85c-drkb4.15394cdc200a5c10   Pod       spec.containers{azure-dl}   Normal    Pulled                kubelet, aks-nodepool1-13347348-4   Successfully pulled image "fboylu/kerasres152-cpu"
59m         59m          1         azure-dl-56bf5dd85c-drkb4.15394cdc2dedf92b   Pod       spec.containers{azure-dl}   Normal    Created               kubelet, aks-nodepool1-13347348-4   Created container
59m         59m          1         azure-dl.15394cd99af490b5                    Service                               Normal    EnsuredLoadBalancer   service-controller                  Ensured load balancer


Check the logs for the application pod.

In [20]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [21]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-06-18 16:07:37,273 CRIT Supervisor running as root (no user in config file)
2018-06-18 16:07:37,277 INFO supervisord started with pid 1
2018-06-18 16:07:38,279 INFO spawned: 'program_exit' with pid 9
2018-06-18 16:07:38,281 INFO spawned: 'nginx' with pid 10
2018-06-18 16:07:38,283 INFO spawned: 'gunicorn' with pid 11
2018-06-18 16:07:39,377 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-18 16:07:40.050460: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-06-18 16:07:44,055 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-06-18 16:07:59,073 INFO success: gunicorn entered RUNNING state, process has stayed up for > than 20 seconds (startsecs)

243187712/243179624 [==============================] - 2s 0us/step
{"path": "/code/driver.py", "level": "INFO", "message": "

In [45]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   35        35        35           35          2h


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [35]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.158.59   40.121.69.221   80:32698/TCP   1h


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Scaling

In [25]:
#!az aks scale --resource-group=$resource_group --name=$aks_name --node-count 3

In [26]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-1   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-2   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-3   Ready     agent     9d        v1.9.6
aks-nodepool1-13347348-4   Ready     agent     9d        v1.9.6


In [40]:
!kubectl scale --current-replicas=30 --replicas=35 deployment/azure-dl

deployment.extensions "azure-dl" scaled


In [48]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-56bf5dd85c-462vr               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-46z7c               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-5q4z8               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-5s78k               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-7xwgg               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-946br               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-9zvdh               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-drkb4               1/1       Running   0          21h
default       azure-dl-56bf5dd85c-fw5l7               1/1       Running   0          20h
default       azure-dl-56bf5dd85c-gkgmd               1/1       Running   0          20h
default       azure-d

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [49]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [3]:
!az aks delete -n $aks_name -g $resource_group -y

In [4]:
!az group delete --name $resource_group -y

 - Finished ..